In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D,Dense,Input,Flatten
from keras import Model

In [ ]:
import gc
d1={0: 'airplane', 1: 'automobile', 2: 'bird', 3: 'cat', 4: 'deer', 5: 'dog',6: 'frog', 7: 'horse', 8: 'ship', 9: 'truck'}
d2={'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5,'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9}

In [ ]:
trainX = np.load('../input/cifar10trainx/trainX.npy')
trainY = pd.read_csv('../input/cifar-10/trainLabels.csv')
trainY = np.array(trainY,copy = False)
trainY = trainY[:,-1]
trainY = np.array([d2[x] for x in trainY],copy=False)
print(trainX.shape,trainY.shape)

In [ ]:
inputs = Input(shape = (32,32,3))
x = keras.layers.Rescaling(1.0/255)(inputs)
x = keras.layers.RandomFlip("horizontal_and_vertical")(x)
x = keras.layers.RandomRotation(0.2)(x)
x = Conv2D(32,5,activation='selu',padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = Conv2D(32,5,activation='selu')(x)
x = keras.layers.BatchNormalization()(x)
x = Conv2D(32,5,activation='selu',padding='same')(x)
x = keras.layers.BatchNormalization()(x)
x = Conv2D(64,5,activation='selu')(x)
x = keras.layers.BatchNormalization()(x)
x = Conv2D(64,2,activation='selu',strides=(2,2))(x)
x = keras.layers.BatchNormalization()(x)
x = Conv2D(64,5,activation='selu')(x)
x = keras.layers.BatchNormalization()(x)
x = Conv2D(128,5,activation='selu')(x)
x = keras.layers.BatchNormalization()(x)
x = Conv2D(128,4,activation='selu')(x)
x = keras.layers.BatchNormalization()(x)
x = Flatten()(x)
x = Dense(120,activation='selu')(x)
x = keras.layers.Dropout(0.5)(x)
x = Dense(80,activation='selu')(x)
x = keras.layers.Dropout(0.5)(x)
x = Dense(40,activation='selu')(x)
x = keras.layers.Dropout(0.5)(x)
outputs = Dense(10,activation='softmax')(x)
model = Model(inputs=inputs,outputs=outputs,name='Cifar10_CNN')
model.summary()
model.compile(optimizer='adam',metrics=['accuracy'],loss = keras.losses.SparseCategoricalCrossentropy())
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.00001)
model.fit(trainX,trainY, batch_size=32, epochs=200,callbacks=[reduce_lr])

In [ ]:
del trainX
del trainY
gc.collect()

In [ ]:
test = np.load('../input/testcifar10/test.npy')
print(test.shape)

In [ ]:
predictions = model.predict(test)
print(predictions)

In [ ]:
del test
gc.collect()

In [ ]:
predictions = np.argmax(predictions,axis=1)
print(predictions)

In [ ]:
df = pd.DataFrame([[x+1,d1[predictions[x]]] for x in range(len(predictions))])
df.to_csv('submission.csv',index=False,header=['id','label'])